## 1. Setup

Import the flow_core components and configure the path.

In [1]:
import sys
sys.path.insert(0, '/home/stellar/PublicRepo/YMHY/anime_streamer_2_0')

from cores.flow_core import (
    compile_flow,
    Tokenizer,
    Parser,
    Resolver,
    Compiler,
    FlowError,
    TokenType,
)

print("✓ Flow Core imported successfully")

✓ Flow Core imported successfully


## 2. Basic Compilation

A simple example showing the full pipeline from source to compiled output.

In [2]:
source = '''
@greeting |<<<Hello, World!>>>|.
@out |$greeting|.
'''

result = compile_flow(source)
print(result)

00:11:24 I FlowController: Compiling source (full pipeline)
Hello, World!


## 3. Style Examples

FLOW supports various style parameters for formatting output.

### 3.1 Title Style

Use `style.title=<<...>>` to add a header to a node.

In [3]:
source = '''
@out
|style.title=<<Welcome>>
|<<<Content goes here>>>|
|.
'''

print(compile_flow(source))

00:11:24 I FlowController: Compiling source (full pipeline)
# Welcome
Content goes here


### 3.2 Divider Style

Use `style.divider=<<true>>` to add a horizontal rule (`---`) after the node content.

> **Note**: Divider is a boolean flag, not a custom string. It always emits `---`.

In [4]:
source = '''
@section1 |<<<First section>>>|.
@section2 |<<<Second section>>>|.

@out
|style.divider=<<true>>
|$section1
|$section2
|.
'''

print(compile_flow(source))

00:11:24 I FlowController: Compiling source (full pipeline)
First section
Second section
---



## 4. List Examples

FLOW supports 4 list styles: `bullet`, `numbered`, `task`, and `task-done`.

> **Note**: `alpha` and `roman` list styles are NOT implemented. Nested lists with different styles have a known limitation where child list items get re-processed by the parent list style.

### 4.1 Bullet List

In [5]:
source = '''
@out
|style.list=<<bullet>>
|@a |<<<First item>>>|.
|@b |<<<Second item>>>|.
|@c |<<<Third item>>>|.
|.
'''

print(compile_flow(source))

00:11:24 I FlowController: Compiling source (full pipeline)
- First item
- Second item
- Third item


### 4.2 Numbered List

In [6]:
source = '''
@out
|style.list=<<numbered>>
|@step1 |<<<Install dependencies>>>|.
|@step2 |<<<Configure settings>>>|.
|@step3 |<<<Run the application>>>|.
|.
'''

print(compile_flow(source))

00:11:24 I FlowController: Compiling source (full pipeline)
1. Install dependencies
2. Configure settings
3. Run the application


### 4.3 Task List (Unchecked)

In [7]:
source = '''
@out
|style.list=<<task>>
|@task1 |<<<Review the code>>>|.
|@task2 |<<<Write documentation>>>|.
|@task3 |<<<Submit for review>>>|.
|.
'''

print(compile_flow(source))

00:11:24 I FlowController: Compiling source (full pipeline)
- [ ] Review the code
- [ ] Write documentation
- [ ] Submit for review


### 4.4 Task List (Checked)

In [8]:
source = '''
@out
|style.list=<<task-done>>
|@done1 |<<<Set up project>>>|.
|@done2 |<<<Implement feature>>>|.
|@done3 |<<<Deploy to production>>>|.
|.
'''

print(compile_flow(source))

00:11:24 I FlowController: Compiling source (full pipeline)
- [x] Set up project
- [x] Implement feature
- [x] Deploy to production


### 4.5 Nested Lists

Nested lists work correctly - inner list items keep their markers and are indented.

In [9]:
# Nested list example - demonstrates current limitation
# Inner bullet items get re-numbered by the outer list
source = '''
@out
|style.list=<<numbered>>
|@step1 |<<<Install dependencies>>>|.
|@step2 |<<<Configure settings>>>|.
|@step3 |<<<Run the application>>>|.
|.
'''

try:
    result = compile_flow(source)
    print(result)
except FlowError as e:
    print(f"Error: {e}")

00:11:24 I FlowController: Compiling source (full pipeline)
1. Install dependencies
2. Configure settings
3. Run the application


In [25]:
# Nested list example - demonstrates current limitation
# Inner bullet items get re-numbered by the outer list
source = '''
@out|style.list=<<numbered>>

|<<<Install dependencies>>>
    |@_| style.list=<<bullet>>
    |<<<Download packages>>>
    |<<<Set up environment>>>
    |.
|<<<Configure settings>>>
|<<<Run the application>>>
|.
'''

try:
    result = compile_flow(source)
    print(result)
except FlowError as e:
    print(f"Error: {e}")

00:18:34 I FlowController: Compiling source (full pipeline)
1. Install dependencies
  - Download packages
  - Set up environment
2. Configure settings
3. Run the application


## 5. Interactive Cell

Use this cell to experiment with your own FLOW source code.

In [11]:
# Your FLOW source here
source = '''
@greeting |<<<Hello!>>>|.
@out |$greeting|.
'''

try:
    result = compile_flow(source)
    print(result)
except FlowError as e:
    print(f"Error: {e}")

00:11:24 I FlowController: Compiling source (full pipeline)


Hello!


## 6. Pipeline Inspection

Step through each stage of the compiler pipeline to see intermediate results.

In [12]:
source = '''
@greeting |style.title=<<Welcome>>|<<<Hello, World!>>>|.
@out |$greeting|.
'''

print("=" * 50)
print("STAGE 1: TOKENIZATION")
print("=" * 50)

tokenizer = Tokenizer()
tokens = tokenizer.tokenize(source)

for token in tokens:
    if token.type != TokenType.EOF:
        print(f"  L{token.line}:C{token.column:02} {token.type.name:15} {token.value!r}")

print(f"\nTotal: {len(tokens)} tokens")

STAGE 1: TOKENIZATION
  L2:C01 NODE_DEF        'greeting'
  L2:C11 PIPE            '|'
  L2:C12 IDENTIFIER      'style.title'
  L2:C23 ASSIGN          '='
  L2:C24 STRING_PRESERVE 'Welcome'
  L2:C35 PIPE            '|'
  L2:C36 STRING_TRIM     'Hello, World!'
  L2:C55 DOT_END         '|.'
  L3:C01 NODE_DEF        'out'
  L3:C06 PIPE            '|'
  L3:C07 NODE_REF        'greeting'
  L3:C16 DOT_END         '|.'

Total: 13 tokens


In [13]:
print("=" * 50)
print("STAGE 2: PARSING")
print("=" * 50)

parser = Parser()
flow_file = parser.parse(tokens)

print(f"\nImports: {len(flow_file.imports)}")
print(f"Nodes: {list(flow_file.nodes.keys())}")
print(f"Assignments: {len(flow_file.assignments)}")
print(f"Out node: {'@' + flow_file.out_node.id if flow_file.out_node else 'None'}")

print("\n--- Node Details ---")
for node_id, node in flow_file.nodes.items():
    print(f"\n@{node_id} (layer={node.layer}):")
    if node.params.style and node.params.style.title:
        print(f"  style.title: {node.params.style.title!r}")
    print(f"  content: {node.content}")

STAGE 2: PARSING

Imports: 0
Nodes: ['greeting', 'out']
Assignments: 0
Out node: @out

--- Node Details ---

@greeting (layer=0):
  style.title: 'Welcome'
  content: ['Hello, World!']

@out (layer=0):
  content: [NodeRef($greeting)]


In [14]:
print("=" * 50)
print("STAGE 3: RESOLUTION")
print("=" * 50)

resolver = Resolver()
resolved = resolver.resolve(flow_file)

print(f"\nResolved nodes: {list(resolved.nodes.keys())}")
print(f"Entry point: @{resolved.out_node.id}")

print("\n--- Resolved Content ---")
for node_id, node in resolved.nodes.items():
    print(f"\n@{node_id}:")
    print(f"  content: {node.content}")

STAGE 3: RESOLUTION

Resolved nodes: ['greeting', 'out']
Entry point: @out

--- Resolved Content ---

@greeting:
  content: ['Hello, World!']

@out:
  content: [NodeRef($greeting)]


In [15]:
print("=" * 50)
print("STAGE 4: COMPILATION")
print("=" * 50)

compiler = Compiler()
output = compiler.compile(resolved)

print("\n--- Final Output ---")
print(output)

STAGE 4: COMPILATION

--- Final Output ---
# Welcome
Hello, World!


## 7. More Complex Examples

### 7.1 Nested Nodes with Slots

In [16]:
source = '''
# Nested node structure with inline slots
@document
|style.title=<<My Document>>
|@header |style.title=<<Header>>|<<<Document header content>>>|.
|@body |<<<Main body content goes here>>>|.
|@footer |<<<Footer with copyright>>>|.
|.

@out |$document|.
'''

print(compile_flow(source))

00:11:24 I FlowController: Compiling source (full pipeline)
# My Document
## Header
Document header content
Main body content goes here
Footer with copyright


### 7.2 Node References

In [17]:
source = '''
# Reusable nodes with references
@greeting |<<<Hello>>>|.
@name |<<<Alice>>>|.
@punctuation |<<<!!!>>>|.

@message |$greeting|<<<, >>>|$name|$punctuation|.

@out |$message|.
'''

print(compile_flow(source))

00:11:24 I FlowController: Compiling source (full pipeline)
Hello
,
Alice
!!!
